# <center><b>NCAA March Madness 2022 Notebook</b></center>

Special thanks to the following notebook:

https://www.kaggle.com/theoviel/ncaa-starter-the-simpler-the-better

Table of Contents:
    
    1.) Loading Necessities
        a.) Libraries
        b.) Dataset
    2.) Exploratory Data Analysis
        a.) Seeds
        b.) Season Results
        c.) Tourney Results
        d.) Team Statistics
        e.) Computer Ratings
    3.) Feature Engineering
        a.) Seeds
        b.) Team Statistics
        c.) Computer Ratings
        d.) Sabermetrics
    4.) Create Train and Test Sets
        a.) Add Symmetrical
        b.) Scoring Margins
        c.) Target Variable
    5.) Modeling and Verifying Model
        a.) Scale Variables
        b.) Logistic Regression
        c.) XGBoost
        d.) LightGBM
        e.) XGBoostRF
        f.) Combined Model
    6.) Creating Submission
        a.) Predicting on Test Set
        b.) Preparing for Automatic Submission

# <h2>1.) Loading Necessities</h2>

# <h3>1A.) Load Libraries</h3>

In [ ]:
import os
import re
import sklearn
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import xgboost as xgb
import optuna
from collections import Counter
from sklearn.ensemble import StackingClassifier 
from sklearn.metrics import *
from sklearn.linear_model import *
from sklearn.metrics import classification_report
from sklearn.model_selection import *
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import QuantileTransformer
from sklearn.linear_model import LogisticRegression
from scipy.stats import skew, kurtosis

pd.set_option('display.max_columns', None)

# <h3>1B.) Load Dataset</h3>

In [ ]:
DATA_PATH = '../input/mens-march-mania-2022/MDataFiles_Stage2/'
# DATA_PATH = '../input/ncaaw-march-mania-2022/'
# DATA_PATH_M = '../input/ncaam-march-mania-2022/'

for filename in os.listdir(DATA_PATH):
    print(filename)

# <h2>2.) Exploratory Data Analysis</h2>

# <h3>2A.) Seeds</h3>

The Tournament Selection Committee seeds every team in the NCAA Tournament ranging from 1 (the best teams) to 16 (the worst ones). The tournament started with 8 teams in 1939 and has since expanded to 68 teams today (4 play-in games). On Selection Sunday (March 13, 2022), the committee releases the bracket with the seeds.

* First character : Region (W, X, Y, or Z)
* Next two digits : Seed within the region (01 to 16)
* Last character (optional): Distinguishes teams between play-ins ( a or b)

In [ ]:
df_seeds = pd.read_csv(DATA_PATH + 'MNCAATourneySeeds.csv')
df_seeds.head()

# <h3>2B.) Season Results</h3>

In [ ]:
df_season_results = pd.read_csv(DATA_PATH + "MRegularSeasonDetailedResults.csv")
# df_season_results = pd.read_csv(DATA_PATH + "WRegularSeasonCompactResults.csv")
df_season_results.drop(['NumOT', 'WLoc'], axis=1, inplace=True)

In [ ]:
# df_tourney_results = pd.read_csv(DATA_PATH + 'MNCAATourneyDetailedResults.csv')
# df_tourney_results.drop(['NumOT', 'WLoc'], axis=1, inplace=True)

In [ ]:
import pandas as pd
import numpy as np

def concat_row(r):
    if r['WTeamID'] < r['LTeamID']:
        res = str(r['Season'])+"_"+str(r['WTeamID'])+"_"+str(r['LTeamID'])
    else:
        res = str(r['Season'])+"_"+str(r['LTeamID'])+"_"+str(r['WTeamID'])
    return res

# Delete leaked from train
def delete_leaked_from_df_train(df_train, df_test):
    df_train['Concats'] = df_train.apply(concat_row, axis=1)
    df_train_duplicates = df_train[df_train['Concats'].isin(df_test['ID'].unique())]
    df_train_idx = df_train_duplicates.index.values
    df_train = df_train.drop(df_train_idx)
    df_train = df_train.drop('Concats', axis=1)

    return df_train 

def read_data(inFile, sep=','):
    df_op = pd.read_csv(filepath_or_buffer=inFile, low_memory=False, encoding='utf-8', sep=sep)
    return df_op

PATH = "../input/mens-march-mania-2022/MDataFiles_Stage1/"
df_test = read_data(PATH+"MSampleSubmissionStage1.csv")
df_train = read_data(PATH+"MNCAATourneyCompactResults.csv")

print("SIZE TRAIN BEFORE :")
print(df_train.shape)
df_train = delete_leaked_from_df_train(df_train, df_test)

print("SIZE TRAIN AFTER :")
print(df_train.shape)

In [ ]:
df_train

In [ ]:
# df_tourney_results

In [ ]:
df_season_results['ScoreMargin'] = df_season_results['WScore'] - df_season_results['LScore']

In [ ]:
df_season_results.head()

In [ ]:
sabermetrics = pd.DataFrame()

sabermetrics['Season'] = df_season_results['Season']
sabermetrics['WTeamID'] = df_season_results['WTeamID']
sabermetrics['LTeamID'] = df_season_results['LTeamID']

# Number of Possessions
sabermetrics['WPossessions'] = (df_season_results['WFGA'] - df_season_results['WOR']) + df_season_results['WTO'] + .44 * df_season_results['WFTA']
sabermetrics['LPossessions'] = (df_season_results['LFGA'] - df_season_results['LOR']) + df_season_results['LTO'] + .44 * df_season_results['LFTA']

df_season_results['WPossessions'] = sabermetrics['WPossessions']
df_season_results['LPossessions'] = sabermetrics['LPossessions']

# Points Per Possession
sabermetrics['WPtsPerPoss'] = df_season_results['WScore'] / df_season_results['WPossessions']
sabermetrics['LPtsPerPoss'] = df_season_results['LScore'] / df_season_results['LPossessions']

# Effective Field Goal Percentage
sabermetrics['WEffectiveFGPct'] = ((df_season_results['WScore'] - df_season_results['WFTM']) / 2) / df_season_results['WFGA']
sabermetrics['LEffectiveFGPct'] = ((df_season_results['LScore'] - df_season_results['LFTM']) / 2) / df_season_results['LFGA']

# Percentage of Field Goals Assisted
sabermetrics['WAssistRate'] = df_season_results['WAst'] / df_season_results['WFGM']
sabermetrics['LAssistRate'] = df_season_results['LAst'] / df_season_results['LFGM']

# Offensive Rebound Percentage
sabermetrics['WOReboundPct'] = df_season_results['WOR'] / (df_season_results['WFGA'] - df_season_results['WFGM'])
sabermetrics['LOReboundPct'] = df_season_results['LOR'] / (df_season_results['LFGA'] - df_season_results['LFGM'])

# Defensive Rebound Percentage
sabermetrics['WDReboundPct'] = df_season_results['WDR'] / (df_season_results['LFGA'] - df_season_results['LFGM'])
sabermetrics['LDReboundPct'] = df_season_results['LDR'] / (df_season_results['WFGA'] - df_season_results['WFGM'])

# Assist to Turnover Ratio
sabermetrics['WATORatio'] = df_season_results['WAst'] / df_season_results['WTO']
sabermetrics['LATORatio'] = df_season_results['LAst'] / df_season_results['LTO']

# Turnover Rate
sabermetrics['WTORate'] = df_season_results['WTO'] / df_season_results['WPossessions']
sabermetrics['LTORate'] = df_season_results['LTO'] /  df_season_results['LPossessions']

# Percentage of Shots Beyond the Arc
sabermetrics['WBArcPct'] = df_season_results['WFGA3'] / df_season_results['WFGA']
sabermetrics['LBArcPct'] = df_season_results['LFGA3'] /  df_season_results['LFGA']

# Free Throw Rate
sabermetrics['WFTRate'] = df_season_results['WFTA'] / df_season_results['WFGA']
sabermetrics['LFTRate'] = df_season_results['LFTA'] /  df_season_results['LFGA']


In [ ]:
winning_columns = sabermetrics[[col for col in sabermetrics.columns if col[0] == 'W']]
losing_columns = sabermetrics[[col for col in sabermetrics.columns if col[0] == 'L']]
winning_columns.loc[:, 'Season'] = sabermetrics['Season']
losing_columns.loc[:, 'Season'] = sabermetrics['Season']

In [ ]:
winning_columns.groupby(['Season', 'WTeamID']).mean()


In [ ]:
losing_columns.groupby(['Season', 'LTeamID']).mean()

In [ ]:
sabermetrics_season_w = winning_columns.groupby(['Season', 'WTeamID']).count().reset_index()[['Season', 'WTeamID']].rename(columns={"WTeamID": "TeamID"})
sabermetrics_season_l = losing_columns.groupby(['Season', 'LTeamID']).count().reset_index()[['Season', 'LTeamID']].rename(columns={"LTeamID": "TeamID"})

# <h3>Compute Wins, Losses, ScoreMarginWin, ScoreMarginLoss</h3>

In [ ]:
num_win = df_season_results.groupby(['Season', 'WTeamID']).count()
num_win = num_win.reset_index()[['Season', 'WTeamID', 'DayNum']].rename(columns={"DayNum": "NumWins", "WTeamID": "TeamID"}).fillna(0)

In [ ]:
num_win.isna().sum()

In [ ]:
num_loss = df_season_results.groupby(['Season', 'LTeamID']).count()
num_loss = num_loss.reset_index()[['Season', 'LTeamID', 'DayNum']].rename(columns={"DayNum": "NumLosses", "LTeamID": "TeamID"}).fillna(0)

In [ ]:
win_score_margin = df_season_results.groupby(['Season', 'WTeamID']).mean().reset_index()
win_score_margin = win_score_margin[['Season', 'WTeamID', 'ScoreMargin']].rename(columns={"ScoreMargin": "AvgWinningScoreMargin", "WTeamID": "TeamID"}).fillna(0)

In [ ]:
lose_score_margin = df_season_results.groupby(['Season', 'LTeamID']).mean().reset_index()
lose_score_margin = lose_score_margin[['Season', 'LTeamID', 'ScoreMargin']].rename(columns={"ScoreMargin": "AvgLosingScoreMargin", "LTeamID": "TeamID"}).fillna(0)

# <h3>Merge the results together</h3>

In [ ]:
df_features_season_w = df_season_results.groupby(['Season', 'WTeamID']).count().reset_index()[['Season', 'WTeamID']].rename(columns={"WTeamID": "TeamID"})
df_features_season_l = df_season_results.groupby(['Season', 'LTeamID']).count().reset_index()[['Season', 'LTeamID']].rename(columns={"LTeamID": "TeamID"})

In [ ]:
df_features_season = pd.concat([df_features_season_w, df_features_season_l], axis=0).drop_duplicates().sort_values(['Season', 'TeamID']).reset_index(drop=True)

In [ ]:
df_features_season = df_features_season.merge(num_win, on=['Season', 'TeamID'], how='left')
df_features_season = df_features_season.merge(num_loss, on=['Season', 'TeamID'], how='left')
df_features_season = df_features_season.merge(win_score_margin, on=['Season', 'TeamID'], how='left')
df_features_season = df_features_season.merge(lose_score_margin, on=['Season', 'TeamID'], how='left')

In [ ]:
df_features_season

In [ ]:
df_features_season['NumWins'] = df_features_season['NumWins'].fillna(0)
df_features_season['NumLosses'] = df_features_season['NumLosses'].fillna(0)
df_features_season['AvgWinningScoreMargin'] = df_features_season['AvgWinningScoreMargin'].fillna(0)
df_features_season['AvgLosingScoreMargin'] = df_features_season['AvgLosingScoreMargin'].fillna(0)

In [ ]:
df_features_season['WinPercentage'] = df_features_season['NumWins'] / (df_features_season['NumWins'] + df_features_season['NumLosses'])
df_features_season['AvgScoringMargin'] = (
    (df_features_season['NumWins'] * df_features_season['AvgWinningScoreMargin'] - 
    df_features_season['NumLosses'] * df_features_season['AvgLosingScoreMargin'])
    / (df_features_season['NumWins'] + df_features_season['NumLosses'])
)

In [ ]:
df_features_season.drop(['AvgWinningScoreMargin', 'AvgLosingScoreMargin'], axis=1, inplace=True)

In [ ]:
df_features_season.isna().sum()

In [ ]:
df_features_season

# <h3>2C.) Tourney Results</h3>


In [ ]:
df_tourney_results = pd.read_csv(DATA_PATH + "MNCAATourneyDetailedResults.csv")
df_tourney_results.drop(['NumOT', 'WLoc'], axis=1, inplace=True)

In [ ]:
df_tourney_results.head()

# <h3>2D.) Computer Ratings</h3>

In [ ]:
df_massey = pd.read_csv(DATA_PATH + "MMasseyOrdinals_thruDay128.csv")
df_massey = df_massey[df_massey['RankingDayNum'] == 128].drop('RankingDayNum', axis=1).reset_index(drop=True) # use first day of the tournament

In [ ]:
systems = []
for year in range(2003, 2019):
    r = df_massey[df_massey['Season'] == year]
    systems.append(r['SystemName'].unique())
    
all_systems = list(set(list(np.concatenate(systems))))

In [ ]:
common_systems = []  
for system in all_systems:
    common = True
    for system_years in systems:
        if system not in system_years:
            common = False
    if common:
        common_systems.append(system)
        
common_systems

# <h2>3.) Feature Engineering</h2>

In [ ]:
df = df_tourney_results.copy()
df = df[df['Season'] >= 2003].reset_index(drop=True)

df.head()

# <h3>3A.) Seeds</h3>

In [ ]:
df = pd.merge(
    df, 
    df_seeds, 
    how='left', 
    left_on=['Season', 'WTeamID'], 
    right_on=['Season', 'TeamID']
).drop('TeamID', axis=1).rename(columns={'Seed': 'SeedW'})

In [ ]:
df = pd.merge(
    df, 
    df_seeds, 
    how='left', 
    left_on=['Season', 'LTeamID'], 
    right_on=['Season', 'TeamID']
).drop('TeamID', axis=1).rename(columns={'Seed': 'SeedL'})

In [ ]:
df

In [ ]:
def treat_seed(seed):
    return int(re.sub("[^0-9]", "", seed))

In [ ]:
df['SeedW'] = df['SeedW'].apply(treat_seed)
df['SeedL'] = df['SeedL'].apply(treat_seed)

In [ ]:
df.head()

# <h3>3B.) Season Stats</h3>

In [ ]:
df = pd.merge(
    df,
    df_features_season,
    how='left',
    left_on=['Season', 'WTeamID'],
    right_on=['Season', 'TeamID']
).rename(columns={
    'NumWins': 'NumWinsW',
    'NumLosses': 'NumLossesW',
    'AvgWinningScoreMargin': 'AvgWinningScoreMarginW',
    'AvgLosingScoreMargin': 'AvgLosingScoreMarginW',
    'WinPercentage': 'WinPercentageW',
    'AvgScoringMargin': 'AvgScoringMarginW',
}).drop(columns='TeamID', axis=1)

df = pd.merge(
    df,
    df_features_season,
    how='left',
    left_on=['Season', 'LTeamID'],
    right_on=['Season', 'TeamID']
).rename(columns={
    'NumWins': 'NumWinsL',
    'NumLosses': 'NumLossesL',
    'AvgWinningScoreMargin': 'AvgWinningScoreMarginL',
    'AvgLosingScoreMargin': 'AvgLosingScoreMarginL',
    'WinPercentage': 'WinPercentageL',
    'AvgScoringMargin': 'AvgScoringMarginL',
}).drop(columns='TeamID', axis=1)

### 3C.) Computer Ratings

In [ ]:
avg_ranking = df_massey.groupby(['Season', 'TeamID']).mean().reset_index()

df = pd.merge(
     df,
     avg_ranking,
     how='left',
     left_on=['Season', 'WTeamID'],
     right_on=['Season', 'TeamID']
).drop('TeamID', axis=1).rename(columns={'OrdinalRank': 'ComputerRankW'})

df = pd.merge(
    df, 
    avg_ranking, 
    how='left', 
    left_on=['Season', 'LTeamID'], 
    right_on=['Season', 'TeamID']
).drop('TeamID', axis=1).rename(columns={'OrdinalRank': 'ComputerRankL'})

In [ ]:
conferences = pd.read_csv(DATA_PATH + 'Conferences.csv')
team_conferences = pd.read_csv(DATA_PATH + 'MTeamConferences.csv')
avg_ranking = df_massey.groupby(['Season', 'TeamID']).mean().reset_index()

In [ ]:
conference_features = avg_ranking.merge(team_conferences, on=['Season', 'TeamID']).groupby(['Season', 'ConfAbbrev'])['OrdinalRank'].agg([np.mean])

# <h3>3D.) Sabermetrics</h3>

In [ ]:
df_season_results.head()

In [ ]:
sabermetrics = pd.DataFrame()

sabermetrics['Season'] = df_season_results['Season']
sabermetrics['WTeamID'] = df_season_results['WTeamID']
sabermetrics['LTeamID'] = df_season_results['LTeamID']

# Number of Possessions
sabermetrics['WPossessions'] = (df_season_results['WFGA'] - df_season_results['WOR']) + df_season_results['WTO'] + .44 * df_season_results['WFTA']
sabermetrics['LPossessions'] = (df_season_results['LFGA'] - df_season_results['LOR']) + df_season_results['LTO'] + .44 * df_season_results['LFTA']

df_season_results['WPossessions'] = sabermetrics['WPossessions']
df_season_results['LPossessions'] = sabermetrics['LPossessions']

# Points Per Possession
sabermetrics['WPtsPerPoss'] = df_season_results['WScore'] / df_season_results['WPossessions']
sabermetrics['LPtsPerPoss'] = df_season_results['LScore'] / df_season_results['LPossessions']

# Effective Field Goal Percentage
sabermetrics['WEffectiveFGPct'] = ((df_season_results['WScore'] - df_season_results['WFTM']) / 2) / df_season_results['WFGA']
sabermetrics['LEffectiveFGPct'] = ((df_season_results['LScore'] - df_season_results['LFTM']) / 2) / df_season_results['LFGA']

# Percentage of Field Goals Assisted
sabermetrics['WAssistRate'] = df_season_results['WAst'] / df_season_results['WFGM']
sabermetrics['LAssistRate'] = df_season_results['LAst'] / df_season_results['LFGM']

# Offensive Rebound Percentage
sabermetrics['WOReboundPct'] = df_season_results['WOR'] / (df_season_results['WFGA'] - df_season_results['WFGM'])
sabermetrics['LOReboundPct'] = df_season_results['LOR'] / (df_season_results['LFGA'] - df_season_results['LFGM'])

# Defensive Rebound Percentage
sabermetrics['WDReboundPct'] = df_season_results['WDR'] / (df_season_results['LFGA'] - df_season_results['LFGM'])
sabermetrics['LDReboundPct'] = df_season_results['LDR'] / (df_season_results['WFGA'] - df_season_results['WFGM'])

# Assist to Turnover Ratio
sabermetrics['WATORatio'] = df_season_results['WAst'] / df_season_results['WTO']
sabermetrics['LATORatio'] = df_season_results['LAst'] / df_season_results['LTO']

# Turnover Rate
sabermetrics['WTORate'] = df_season_results['WTO'] / df_season_results['WPossessions']
sabermetrics['LTORate'] = df_season_results['LTO'] /  df_season_results['LPossessions']

# Percentage of Shots Beyond the Arc
sabermetrics['WBArcPct'] = df_season_results['WFGA3'] / df_season_results['WFGA']
sabermetrics['LBArcPct'] = df_season_results['LFGA3'] /  df_season_results['LFGA']

# Free Throw Rate
sabermetrics['WFTRate'] = df_season_results['WFTA'] / df_season_results['WFGA']
sabermetrics['LFTRate'] = df_season_results['LFTA'] /  df_season_results['LFGA']

# Block to Foul Percentage
sabermetrics['WBlockFoul'] = df_season_results['WBlk'] / (df_season_results['WPF'] + df_season_results['WBlk'])
sabermetrics['LBlockFoul'] = df_season_results['LBlk'] / (df_season_results['LPF'] + df_season_results['LBlk'])

# Steal to Foul Percentage
sabermetrics['WStealFoul'] = df_season_results['WStl'] / (df_season_results['WPF'] + df_season_results['WStl'])
sabermetrics['LStealFoul'] = df_season_results['LStl'] / (df_season_results['LPF'] + df_season_results['LStl'])


In [ ]:
sabermetrics

In [ ]:
winning_columns = sabermetrics[[col for col in sabermetrics.columns if col[0] == 'W']]
losing_columns = sabermetrics[[col for col in sabermetrics.columns if col[0] == 'L']]
winning_columns.loc[:, 'Season'] = sabermetrics['Season']
losing_columns.loc[:, 'Season'] = sabermetrics['Season']

In [ ]:
winning_sabermetrics = winning_columns.groupby(['Season', 'WTeamID']).mean()
losing_sabermetrics = losing_columns.groupby(['Season', 'LTeamID']).mean()

In [ ]:
winning_sabermetrics

In [ ]:
losing_sabermetrics

In [ ]:
winning_sabermetrics = winning_sabermetrics \
                        .reset_index() \
                        .merge(df_features_season[['Season', 'TeamID', 'NumWins']], left_on=['Season', 'WTeamID'], right_on=['Season', 'TeamID'], how='left') \
                        .set_index(['Season', 'WTeamID']) \
                        .drop(['TeamID'], axis=1)

losing_sabermetrics = losing_sabermetrics \
                        .reset_index() \
                        .merge(df_features_season[['Season', 'TeamID', 'NumLosses']], left_on=['Season', 'LTeamID'], right_on=['Season', 'TeamID'], how='left') \
                        .set_index(['Season', 'LTeamID']) \
                        .drop(['TeamID'], axis=1)

In [ ]:
weighted_sabermetrics_wins = winning_sabermetrics[[col for col in winning_sabermetrics.columns if col[0] == 'W']].multiply(winning_sabermetrics['NumWins'], axis=0)
weighted_sabermetrics_losses = losing_sabermetrics[[col for col in losing_sabermetrics.columns if col[0] == 'L']].multiply(losing_sabermetrics['NumLosses'], axis=0)

weighted_sabermetrics = pd.DataFrame()
weighted_sabermetrics['Possessions'] = (weighted_sabermetrics_wins['WPossessions'] + weighted_sabermetrics_losses['LPossessions']) /  \
                                       (winning_sabermetrics['NumWins'] + losing_sabermetrics['NumLosses'])

combined_df = winning_sabermetrics.reset_index().merge(losing_sabermetrics.reset_index(), left_on=['WTeamID', 'Season'], right_on=['LTeamID', 'Season'], how='outer')

def weighted_metric(metric, df=combined_df.set_index(['Season', 'WTeamID'], inplace=True)):
    """Computes the weighted stat from winning and losing metric"""
        
    weighted_df = ((combined_df[f'W{metric}'].mul(combined_df['NumWins']) + combined_df[f'L{metric}'].mul(combined_df['NumLosses'])) \
    / (combined_df['NumWins'] + combined_df['NumLosses']))
    return weighted_df

In [ ]:
combined_df.reset_index(inplace=True)
combined_df['WTeamID'].fillna(combined_df['LTeamID'], inplace=True)
combined_df['LTeamID'].fillna(combined_df['WTeamID'], inplace=True)
combined_df.set_index(['Season', 'WTeamID'], inplace=True)
combined_df.fillna(0, inplace=True)

In [ ]:
metrics_list = ['Possessions', 'PtsPerPoss', 'EffectiveFGPct', 'AssistRate', 'OReboundPct', 'DReboundPct', 'ATORatio', 'TORate', 'BArcPct', 'FTRate', 'BlockFoul', 'StealFoul']
season_sabermetrics = pd.concat([weighted_metric(metric) for metric in metrics_list], axis=1)
season_sabermetrics.columns=metrics_list

In [ ]:
season_sabermetrics.sort_index(inplace=True)

In [ ]:
season_sabermetrics.index.columns = ['Season', 'TeamID']

In [ ]:
season_sabermetrics.reset_index(inplace=True)

In [ ]:
season_sabermetrics

In [ ]:
df = df.merge(season_sabermetrics, left_on=['Season', 'WTeamID'], right_on=['Season', 'WTeamID'], how='left', suffixes=[None, 'W'])
df = df.merge(season_sabermetrics, left_on=['Season', 'LTeamID'], right_on=['Season', 'WTeamID'], how='left', suffixes=[None, 'L'])

In [ ]:
df

# <h3>3E.) Conference Strength</h3>

In [ ]:
team_conferences

In [ ]:

conferences = pd.read_csv(DATA_PATH + 'Conferences.csv')
team_conferences = pd.read_csv(DATA_PATH + 'MTeamConferences.csv')
avg_ranking = df_massey.groupby(['Season', 'TeamID']).mean().reset_index()
conference_features = avg_ranking.merge(team_conferences, on=['Season', 'TeamID']).groupby(['Season', 'ConfAbbrev'])['OrdinalRank'].agg([np.mean])
conference_computer_ranks = team_conferences.merge(conference_features, left_on=['Season', 'ConfAbbrev'], right_index=True)
df = df.merge(conference_computer_ranks, how='left', left_on=['Season', 'WTeamID'], right_on=['Season', 'TeamID']).drop(['TeamID', 'ConfAbbrev'], axis=1).rename(columns={'mean': 'ConferenceMeanA'})
df = df.merge(conference_computer_ranks, how='left', left_on=['Season', 'LTeamID'], right_on=['Season', 'TeamID']).drop(['TeamID', 'ConfAbbrev'], axis=1).rename(columns={'mean': 'ConferenceMeanB'})
df['ConferenceMeanDiff'] = df['ConferenceMeanA'] - df['ConferenceMeanB']

In [ ]:
df

# <h2>4.) Create Train and Test Sets</h2>

# <h3>4A.) Add Symmetrical</h3>

### Add symmetrical
- Right now our data only consists of won matches
- We duplicate our data, get rid of the winner loser 

In [ ]:
def add_losing_matches(win_df):
    win_rename = {
        "WTeamID": "TeamIDA", 
        "WScore" : "ScoreA", 
        "LTeamID" : "TeamIDB",
        "LScore": "ScoreB",
        "SeedW": "SeedA", 
        "SeedL": "SeedB",
        'WinPercentageW' : 'WinPercentageA',
        'WinPercentageL' : 'WinPercentageB',
        'AvgScoringMarginW' : 'AvgScoringMarginA',
        'AvgScoringMarginL' : 'AvgScoringMarginB',
        "ComputerRankW": "ComputerRankA",
        "ComputerRankL": "ComputerRankB",
        'EffectiveFGPct': 'EffectiveFGPctA',
         'PtsPerPoss': 'PtsPerPossA',
         'Possessions': 'PossessionsA',
         'AssistRate': 'AssistRateA',
         'OReboundPct': 'OReboundPctA',
         'DReboundPct': 'DReboundPctA',
         'ATORatio':'ATORatioA', 
         'TORate': 'TORateA',
         'BArcPct': 'BArcPctA',
         'FTRate': 'FTRateA',
         'StealFoul': 'StealFoulA',
         'BlockFoul': 'BlockFoulA',
         'PossessionsL': 'PossessionsB',
         'PtsPerPossL': 'PtsPerPossB',
         'EffectiveFGPctL': 'EffectiveFGPctB',
         'AssistRateL': 'AssistRateB',
         'OReboundPctL': 'OReboundPctB',
         'DReboundPctL': 'DReboundPctB',
         'ATORatioL': 'ATORatioB',
         'TORateL': 'TORateB',
         'BArcPctL': 'BArcPctB',
         'FTRateL': 'FTRateB',
         'StealFoulL': 'StealFoulB',
         'BlockFoulL': 'BlockFoulB'
     }
    
    lose_rename = {
        "WTeamID": "TeamIDB", 
        "WScore" : "ScoreB", 
        "LTeamID" : "TeamIDA",
        "LScore": "ScoreA",
        "SeedW": "SeedB", 
        "SeedL": "SeedA",
        'WinPercentageW' : 'WinPercentageB',
        'WinPercentageL' : 'WinPercentageA',
        'AvgScoringMarginW' : 'AvgScoringMarginB',
        'AvgScoringMarginL' : 'AvgScoringMarginA',
        "ComputerRankW": "ComputerRankB",
        "ComputerRankL": "ComputerRankA",
        "EffectiveFGPct": "EffectiveFGPctB",
        "EffectiveFGPctL": "EffectiveFGPctA",
        "PtsPerPoss": "PtsPerPossB",
        "PtsPerPossL": "PtsPerPossA",
         'Possessions': 'PossessionsB',
         'AssistRate': 'AssistRateB',
         'OReboundPct': 'OReboundPctB',
         'DReboundPct': 'DReboundPctB',
         'ATORatio':'ATORatioB', 
         'TORate': 'TORateB',
         'BArcPct': 'BArcPctB',
         'FTRate': 'FTRateB',
         'StealFoul': 'StealFoulB',
         'BlockFoul': 'BlockFoulB',
         'PossessionsL': 'PossessionsA',
         'PtsPerPossL': 'PtsPerPossA',
         'EffectiveFGPctL': 'EffectiveFGPctA',
         'AssistRateL': 'AssistRateA',
         'OReboundPctL': 'OReboundPctA',
         'DReboundPctL': 'DReboundPctA',
         'ATORatioL': 'ATORatioA',
         'TORateL': 'TORateA',
         'BArcPctL': 'BArcPctA',
         'FTRateL': 'FTRateA',
         'StealFoulL': 'StealFoulA',
         'BlockFoulL': 'BlockFoulA',
    }
    
    win_df = win_df.copy()
    lose_df = win_df.copy()
    
    win_df = win_df.rename(columns=win_rename)
    lose_df = lose_df.rename(columns=lose_rename)
    
    merged_df = pd.concat([win_df, lose_df], axis=0, sort=False)
    
    return merged_df

In [ ]:
df = add_losing_matches(df)

In [ ]:
df.fillna(0, inplace=True)

# <h3>4E.) Metric Differences</h3>

In [ ]:
df['SeedDiff'] = df['SeedA'] - df['SeedB']
df['WinPercentageDiff'] = df['WinPercentageA'] - df['WinPercentageB']
df['AvgScoringMarginDiff'] = df['AvgScoringMarginA'] - df['AvgScoringMarginB']
df['ComputerRankDiff'] = df['ComputerRankA'] - df['ComputerRankB']

In [ ]:
df

In [ ]:
conferences = pd.read_csv(DATA_PATH + 'Conferences.csv')
team_conferences = pd.read_csv(DATA_PATH + 'MTeamConferences.csv')

In [ ]:
team_conferences.merge(conferences, on='ConfAbbrev', how='left')

# <h3>4F.) Test Data</h3>

In [ ]:
df_test = pd.read_csv(DATA_PATH + "MSampleSubmissionStage2.csv")
df_test['Season'] = df_test['ID'].apply(lambda x: int(x.split('_')[0]))
df_test['TeamIDA'] = df_test['ID'].apply(lambda x: int(x.split('_')[1]))
df_test['TeamIDB'] = df_test['ID'].apply(lambda x: int(x.split('_')[2]))

<h3>Seeds</h3>

In [ ]:
df_test = pd.merge(
    df_test,
    df_seeds,
    how='left',
    left_on=['Season', 'TeamIDA'],
    right_on=['Season', 'TeamID']
).drop('TeamID', axis=1).rename(columns={'Seed': 'SeedA'})

df_test = pd.merge(
    df_test, 
    df_seeds, 
    how='left', 
    left_on=['Season', 'TeamIDB'], 
    right_on=['Season', 'TeamID']
).drop('TeamID', axis=1).rename(columns={'Seed': 'SeedB'})

df_test['SeedA'] = df_test['SeedA'].apply(treat_seed)
df_test['SeedB'] = df_test['SeedB'].apply(treat_seed)

<h3>Season Stats</h3>

In [ ]:
df_test = pd.merge(
    df_test,
    df_features_season,
    how='left',
    left_on=['Season', 'TeamIDA'],
    right_on=['Season', 'TeamID']
).rename(columns={
    'NumWins': 'NumWinsA',
    'NumLosses': 'NumLossesA',
    'AvgWinningScoreMargin': 'AvgWinningScoreMarginA',
    'AvgLosingScoreMargin': 'AvgLosingScoreMarginA',
    'WinPercentage': 'WinPercentageA',
    'AvgScoringMargin': 'AvgScoringMarginA',
    'EffectiveFGPct': 'EffectiveFGPctA',
    'PtsPerPoss': 'PtsPerPossA',
    'Possessions': 'PossessionsA',
    'AssistRate': 'AssistRateA',
    'OReboundPct': 'OReboundPctA',
    'DReboundPct': 'DReboundPctA',
    'ATORatio':'ATORatioA', 
    'TORate': 'TORateA',
    'BArcPct': 'BArcPctA',
    'FTRate': 'FTRateA',
    'StealFoul': 'StealFoulA',
    'BlockFoul': 'BlockFoulA'
}).drop(columns='TeamID', axis=1)

df_test = pd.merge(
    df_test,
    df_features_season,
    how='left',
    left_on=['Season', 'TeamIDB'],
    right_on=['Season', 'TeamID']
).rename(columns={
    'NumWins': 'NumWinsB',
    'NumLosses': 'NumLossesB',
    'AvgWinningScoreMargin': 'AvgWinningScoreMarginB',
    'AvgLosingScoreMargin': 'AvgLosingScoreMarginB',
    'WinPercentage': 'WinPercentageB',
    'AvgScoringMargin': 'AvgScoringMarginB',
    'PossessionsL': 'PossessionsB',
    'PtsPerPossL': 'PtsPerPossB',
    'EffectiveFGPctL': 'EffectiveFGPctB',
    'AssistRateL': 'AssistRateB',
    'OReboundPctL': 'OReboundPctB',
    'DReboundPctL': 'DReboundPctB',
    'ATORatioL': 'ATORatioB',
    'TORateL': 'TORateB',
    'BArcPctL': 'BArcPctB',
    'FTRateL': 'FTRateB',
    'StealFoulL': 'StealFoulB',
    'BlockFoulL': 'BlockFoulB'
    
}).drop(columns='TeamID', axis=1)

# <h3>Computer Ratings</h3>

In [ ]:
df_test = pd.merge(
    df_test,
    avg_ranking,
    how='left',
    left_on=['Season', 'TeamIDA'],
    right_on=['Season', 'TeamID']
).drop('TeamID', axis=1).rename(columns={'OrdinalRank': 'ComputerRankA'})

df_test = pd.merge(
    df_test,
    avg_ranking,
    how='left',
    left_on=['Season', 'TeamIDB'],
    right_on=['Season', 'TeamID']
).drop('TeamID', axis=1).rename(columns={'OrdinalRank': 'ComputerRankB'})

In [ ]:
df_test

# <h3>Differences</h3>

In [ ]:
df_test['SeedDiff'] = df_test['SeedA'] - df_test['SeedB']
df_test['WinPercentageDiff'] = df_test['WinPercentageA'] - df_test['WinPercentageB']
df_test['AvgScoringMarginDiff'] = df_test['AvgScoringMarginA'] - df_test['AvgScoringMarginB']
df_test['ComputerRankDiff'] = df_test['ComputerRankA'] - df_test['ComputerRankB']

# <h3>Sabermetrics</h3>

In [ ]:
df_test = df_test.merge(season_sabermetrics, left_on=['Season', 'TeamIDA'], right_on=['Season', 'WTeamID'], how='left', suffixes=[None, 'A'])
df_test = df_test.merge(season_sabermetrics, left_on=['Season', 'TeamIDB'], right_on=['Season', 'WTeamID'], how='left', suffixes=[None, 'B'])

In [ ]:
df_test = df_test.merge(conference_computer_ranks, how='left', left_on=['Season', 'TeamIDA'], right_on=['Season', 'TeamID']).drop(['TeamID', 'ConfAbbrev'], axis=1).rename(columns={'mean': 'ConferenceMeanA'})
df_test = df_test.merge(conference_computer_ranks, how='left', left_on=['Season', 'TeamIDB'], right_on=['Season', 'TeamID']).drop(['TeamID', 'ConfAbbrev'], axis=1).rename(columns={'mean': 'ConferenceMeanB'})
df_test['ConferenceMeanDiff'] = df_test['ConferenceMeanA'] - df_test['ConferenceMeanB']

In [ ]:
df_test

In [ ]:
df_test = df_test.rename(columns={'EffectiveFGPct': 'EffectiveFGPctA',
                                 'PtsPerPoss': 'PtsPerPossA',
                                 'Possessions': 'PossessionsA',
                                 'AssistRate': 'AssistRateA',
                                 'OReboundPct': 'OReboundPctA',
                                 'DReboundPct': 'DReboundPctA',
                                 'ATORatio':'ATORatioA', 
                                 'TORate': 'TORateA',
                                 'BArcPct': 'BArcPctA',
                                 'FTRate': 'FTRateA',
                                 'StealFoul': 'StealFoulA',
                                 'BlockFoul': 'BlockFoulA',
                                 'PossessionsL': 'PossessionsB',
                                 'PtsPerPossL': 'PtsPerPossB',
                                 'EffectiveFGPctL': 'EffectiveFGPctB',
                                 'AssistRateL': 'AssistRateB',
                                 'OReboundPctL': 'OReboundPctB',
                                 'DReboundPctL': 'DReboundPctB',
                                 'ATORateL': 'ATORatioB',
                                 'TORateL': 'TORateB',
                                 'BArcPctL': 'BArcPctB',
                                 'FTRateL': 'FTRateB',
                                 'StealFoulL': 'StealFoulB',
                                 'BlockFoulL': 'BlockFoulB'})

# <h3>4F.) Target Variable</h3>

In [ ]:
df['ScoreDiff'] = df['ScoreA'] - df['ScoreB']
df['WinA'] = (df['ScoreDiff'] > 0).astype(int)

# <h2>5.) Modeling and Verifying Model</h2>

# <h3>5A.) Scaling Variables<h3>

In [ ]:
features = [
    "SeedA", 
    "SeedB",
    'WinPercentageA',
    'WinPercentageB',
    'AvgScoringMarginA',
    'AvgScoringMarginB',
    "ComputerRankA",
    "ComputerRankB",
    'PossessionsA',
    'PtsPerPossA',
    'EffectiveFGPctA',
    'AssistRateA',
    'OReboundPctA',
    'DReboundPctA',
    'ATORatioA', 
    'TORateA',
    'BArcPctA',
    'FTRateA',
    'StealFoulA',
    'BlockFoulA',
    'PossessionsB',
    'PtsPerPossB',
    'EffectiveFGPctB',
    'AssistRateB',
    'OReboundPctB',
    'DReboundPctB',
    'ATORatioB',
    'TORateB',
    'BArcPctB',
    'FTRateB',
    'StealFoulB',
    'BlockFoulB',
    'WinPercentageDiff',
    'ComputerRankDiff'
]

target = ['WinA']

In [ ]:

def rescale(features, df_train, df_val, df_test=None):
    min_ = df_train[features].min()
    max_ = df_train[features].max()
    
    df_train[features] = (df_train[features] - min_) / (max_ - min_)
    df_val[features] = (df_val[features] - min_) / (max_ - min_)
    
    if df_test is not None:
        df_test[features] = (df_test[features] - min_) / (max_ - min_)
        
    return df_train, df_val, df_test


In [ ]:

def standard_scale(features, df_train, df_val, df_test=None):
    pt = PowerTransformer(method='yeo-johnson')
    df_train[features] = pt.fit_transform(df_train[features])
    df_val[features] = pt.transform(df_val[features])
    
    if df_test is not None:
        df_test[features] = pt.transform(df_test[features])
    
    return df_train, df_val, df_test


In [ ]:
df_test['ComputerRankDiff']

# <h3>5B.) Modeling</h3>

In [ ]:
season = 2021
df_train = df[df['Season'] <= season].copy()
df_val = df[df['Season'] == season].copy()        
rescaled = standard_scale(features, df_train, df_val)
X_train = rescaled[0][features]
X_validation = rescaled[1][features]
y_train = df_train['WinA'].values
y_validation = df_val['WinA'].values

In [ ]:

Cs = [.0001, .0003, .001, .003, .01, .03, .1, .3, 1, 3, 10, 30, 100, 300, 1000]
scores = []
for c in Cs:
    logistic = LogisticRegression(C=c)
    logistic.fit(X_train, y_train)
    predictions = logistic.predict_proba(X_validation)
    scores.append(logistic.score(X_validation, y_validation))
max_C = Cs[np.argmax(scores)]

In [ ]:
max_C

In [ ]:
def objective(trial):
    xgb_params = dict(
        max_depth=trial.suggest_int("max_depth", 10, 64),
        learning_rate=trial.suggest_float("learning_rate", 1e-5, 1e-1, log=True),
        n_estimators=trial.suggest_int("n_estimators", 10, 80),
        min_child_weight=trial.suggest_int("min_child_weight", 10, 20),
        colsample_bytree=trial.suggest_float("colsample_bytree", 0.01, 1),
        subsample=trial.suggest_float("subsample", 0.2, 1),
        reg_alpha=trial.suggest_float("reg_alpha", 1e-4, 1e-1, log=True),
        reg_lambda=trial.suggest_float("reg_lambda", 1e-4, 1e2, log=True),
        gamma=trial.suggest_float("gamma", 1e-6, 1, log=True)
    )
    xg_cl = xgb.XGBClassifier(**xgb_params, objective='binary:logistic', eval_metric='logloss', use_label_encoder=False)
    xg_cl.fit(X_train, y_train)
    return xg_cl.score(X_validation, y_validation)

study = optuna.create_study()
study.optimize(objective, n_trials=200)
xgb_params = study.best_params


In [ ]:
from optuna.visualization import plot_slice
plot_slice(study)

In [ ]:
from lightgbm import LGBMClassifier
lightgbmc_model = LGBMClassifier(boosting_type='gbdt', objective='binary', random_state=42)

In [ ]:

def objective(trial):
    lgbm_params = {
        'reg_alpha': trial.suggest_float("reg_alpha", 1e-4, 1e2, log=True),
        'reg_lambda': trial.suggest_float("reg_lambda", 1e-4, 1e2, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 200, 500),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 25),
        'max_depth': trial.suggest_int('max_depth', 5, 64),
        'learning_rate': trial.suggest_float("learning_rate", 1e-4, 1e-2, log=True),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 0.9),
        'n_estimators': trial.suggest_int('n_estimators', 10, 80),
    }
    
    lgbm_cl = LGBMClassifier(**lgbm_params, boosting_type='gbdt', objective='binary', random_state=42)
    lgbm_cl.fit(X_train, y_train)
    return lgbm_cl.score(X_validation, y_validation)

study = optuna.create_study()
study.optimize(objective, n_trials=100)
lgbm_params = study.best_params
plot_slice(study)


In [ ]:
def objective(trial):
    xgb_rf_params = dict(
        max_depth=trial.suggest_int("max_depth", 10, 64),
        eta=trial.suggest_float("eta", 1e-3, .3),
        colsample_bytree=trial.suggest_float("colsample_bytree", .01, 1),
        colsample_bynode=trial.suggest_float("colsample_bynode", .01, 1),
        colsample_bylevel=trial.suggest_float("colsample_bylevel", .01, 1),
        reg_alpha=trial.suggest_float("reg_alpha", 1e-4, 1e2, log=True),
        reg_lambda=trial.suggest_float("reg_lambda", 1e-4, 1e2, log=True),
        gamma=trial.suggest_float("gamma", 1e-6, 1, log=True),
        scale_pos_weight=1
    )
    xg_cl = xgb.XGBRFClassifier(**xgb_rf_params, objective='binary:logistic', eval_metric='logloss', use_label_encoder=False)
    xg_cl.fit(X_train, y_train)
    return xg_cl.score(X_validation, y_validation)

study = optuna.create_study()
study.optimize(objective, n_trials=200)
xgb_rf_params = study.best_params


In [ ]:
xgb_rf_params = study.best_params
plot_slice(study)


<h3>5C.) Cross Validation</h3>

In [ ]:
def kfold_reg(df, df_test_=None, plot=False, verbose=0, mode="reg"):
    seasons = df['Season'].unique()
    cvs = []
    pred_tests = []
    target = "ScoreDiff" if mode == "reg" else "WinA"
    
    for season in seasons[13:]:
        if verbose:
            print(f'\nValidating on season {season}')
        
        df_train = df[df['Season'] < season].copy()
        df_val = df[df['Season'] == season].copy()
        df_test = df_test_.copy()
        
        df_train, df_val, df_test = rescale(features, df_train, df_val, df_test)
        
        if mode == "reg":
            model = ElasticNet(alpha=1, l1_ratio=0.5)
        else:
            model0 = LogisticRegression(C=max_C)
            model1 = xgb.XGBClassifier(**xgb_params, objective='binary:logistic', eval_metric='logloss', use_label_encoder=False)
            model2 = xgb.XGBRFClassifier(**xgb_rf_params, objective='binary:logistic', eval_metric='logloss', use_label_encoder=False)
            model3 = LGBMClassifier(**lgbm_params, boosting_type='gbdt', objective='binary', random_state=42)
            model = StackingClassifier(estimators=[('XGB', model1), ('XGBRF', model2), ('LGBM', model3)], 
                                                   final_estimator=SGDClassifier(loss='modified_huber', class_weight='balanced', validation_fraction=.2, fit_intercept=True, alpha=.01))
            
        model.fit(df_train[features], df_train[target])
        
        if mode == "reg":
            pred = model.predict(df_val[features])
        else:
            pred = model.predict_proba(df_val[features])[:, 1]
            pred = (pred - pred.min()) / (pred.max() - pred.min())

        
        if df_test is not None:
            if mode == "reg":
                pred_test = model.predict(df_test[features])
            else:
                pred_test = model.predict_proba(df_test[features])[:, 1]
            pred_test = (pred_test - pred_test.min()) / (pred_test.max() - pred_test.min())
            
            pred_tests.append(pred_test)
            
        if plot:
            plt.figure(figsize=(15, 6))
            plt.subplot(1, 2, 1)
            plt.scatter(pred, df_val['ScoreDiff'].values, s=5)
            plt.grid(True)
            plt.subplot(1, 2, 2)
            sns.histplot(pred)
            plt.show()
        
        loss = log_loss(df_val['WinA'].values, pred)
        cvs.append(loss)

        if verbose:
            print(f'\t -> Scored {loss:.3f}')
        
    print(f'\n Local CV is {np.mean(cvs):.3f}')
    
    return pred_tests

# <h2>6.) Create Submission</h2>

# <h3>6A.) Predict on Test Set</h3>

In [ ]:
df_test[features]

In [ ]:
df_test[features]

In [ ]:
pred_tests = kfold_reg(df, df_test[features], plot=True, verbose=1, mode="cls")

In [ ]:
import lightgbm
f, ax = plt.subplots(figsize=(16, 8))
model = LGBMClassifier(boosting_type='gbdt', objective='binary', random_state=42)
model.fit(X_train, y_train)
lightgbm.plot_importance(model, ax=ax)
plt.show()

In [ ]:
np.mean(np.mean(pred_tests, 0))

# <h3>6B.) Preparing for Automatic Submission</h3>

In [ ]:
pred_test = np.mean(pred_tests, 0)
sub = df_test[['ID', 'Pred']].copy()
sub['Pred'] = pred_test
sub.to_csv('submission.csv', index=False)

In [ ]:
sns.histplot(sub['Pred'])

In [ ]:
# Create human readable output for tournament brackets

human_sub = sub.copy()
human_sub['Season'] = human_sub['ID'].apply(lambda x: int(x.split('_')[0]))
human_sub['TeamIDA'] = human_sub['ID'].apply(lambda x: int(x.split('_')[1]))
human_sub['TeamIDB'] = human_sub['ID'].apply(lambda x: int(x.split('_')[2]))

team_names = 'MTeams.csv'
teams = pd.read_csv(DATA_PATH + team_names)
team_names_1 = pd.merge(human_sub, teams[['TeamID', 'TeamName']], left_on='TeamIDA', right_on='TeamID')
team_names_2 = pd.merge(team_names_1, teams[['TeamID', 'TeamName']], left_on='TeamIDB', right_on='TeamID')

# To make human predictions, export the following columns into Excel and make a pivot table
team_names_2[['Pred', 'TeamName_x', 'TeamName_y']].to_csv('Train2021.csv')